In [5]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from ipywidgets import FloatLogSlider, FloatSlider, interactive
from tqdm.notebook import tqdm

from generators import GCL, LXM

sns.set()



class Satellite:
    def __init__(self, C=math.inf, K=4, μd=5, μ0=0.5, λ=2):
        """Args:
            C: Capture queue length limit.
            K: onboard storage capacity.
            μd: The download service time of an image is assumed exponentially
                distributed with mean 1/μ_d.
            μ0: The basic image capture service time is exponentially distributed
                with mean 1/μ_0
            λ: Arrival rate of new requests.
        """
        self.K = K
        self.C = C
        self.μd = μd
        self.μ0 = μ0
        self.λ = λ
        self.n = 0
        self.m = 0
        self.gen = LXM()

    @property
    def onboard_storage_capacity(self):
        return self.K

    @property
    def images_on_onboard_storage(self):
        return self.m

    @property
    def pending_requests(self):
        return self.n

    def transition(self):
        ps = []
        nexts = []
        if 0 <= self.n < self.C and 0 <= self.m <= self.K:
            ps.append(self.λ)
            nexts.append((self.n + 1, self.m))
        if 0 < self.n <= self.C and 0 <= self.m < self.K:
            ps.append(self.n * self.μ0)
            nexts.append((self.n - 1, self.m + 1))
        if 0 <= self.n <= self.C and 0 < self.m <= self.K:
            ps.append(self.μd)
            nexts.append((self.n, self.m - 1))

        ps = np.array(ps)
        p = self.gen.generar() * ps.sum()
        try:
            self.n, self.m = nexts[np.digitize(p, np.cumsum(ps))]
        except:
            print(f"{nexts=} {ps=} {p=} {ps.cumsum()} {self.n=} {self.m=}")
            raise


def explore_config(λ, μd, μ0, K, n=100_000):
    s = Satellite(λ=λ, μd=μd, μ0=μ0, K=K)

    history_images_on_board_storage = []
    history_pending_requests = []
    position_history = []

    for _ in tqdm(range(int(n))):
        history_images_on_board_storage.append(s.images_on_onboard_storage)
        history_pending_requests.append(s.pending_requests)
        position_history.append((s.n, s.m))

        s.transition()

    d1 = pd.Series(history_images_on_board_storage).value_counts().sort_index() / n
    d2 = pd.Series(history_pending_requests).value_counts().sort_index() / n

    f, (ax0, ax1, ax2) = plt.subplots(
        nrows=1,
        ncols=3,
        dpi=100,
        figsize=(24, 8),
        gridspec_kw={"width_ratios": [1, 1, 2]},
    )

    sns.lineplot(x=d1.index, y=d1.values, ax=ax0, linestyle="--")
    sns.scatterplot(x=d1.index, y=d1.values, ax=ax0, linestyle="--")
    ax0.set_title("Images on board")
    ax0.set_ylabel("# times")
    ax0.set_xlabel("# images on board")

    sns.lineplot(x=d2.index, y=d2.values, ax=ax1, linestyle="--")
    sns.scatterplot(x=d2.index, y=d2.values, ax=ax1, linestyle="--")
    ax1.set_title("Pending requests")
    ax1.set_ylabel("# times")
    ax1.set_xlabel("# pending requests")

    m = (
        pd.DataFrame(
            {
                "n": [x[0] for x in position_history],
                "m": [x[1] for x in position_history],
            }
        )
        .assign(v=1)
        .pivot_table(index="n", columns="m", values="v", aggfunc="sum", fill_value=0)
        / n
    )
    sns.heatmap(m, ax=ax2, square=False, fmt=".4f", annot=True)
    ax2.set_title("Steady point probabilities")

    f.suptitle(f"N={n}  $\lambda$={λ}  $\mu_d$={μd}  $\mu_0$={μ0}  K={K}")

    plt.show()

In [8]:
inter = interactive(
    explore_config,
    {'manual': True},
    λ=FloatSlider(value=2, min=0.1, max=10, step=0.1, description="$\lambda$"),
    μd=FloatSlider(value=5, min=0.01, max=10, step=0.01, description="$\mu_d$"),
    μ0=FloatSlider(value=0.5, min=0.01, max=10, step=0.01, description="$\mu_0$"),
    K=FloatSlider(value=4, min=1, max=10, step=1, description="$K$"),
    n=FloatLogSlider(value=10_000, base=10, min=2, max=8, step=1, description="Iterations", readout_format="d"),
)
display(inter)

interactive(children=(FloatSlider(value=2.0, description='$\\lambda$', max=10.0, min=0.1), FloatSlider(value=5…